# Task 2: A Sample of Owners

Connect to GBQ

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import os
import csv
import random

In [ ]:
service_path = "C:\\Users\\mikes\\OneDrive\\Documents\\Applied Data Analytics\\WedgeProject2020_MikeS\\ada-wedge-project\\"
service_file = 'wedge2020-e941b1559b9f.json' #authentication information  
gbq_proj_id = 'wedge2020'  
gbq_dataset_id = 'wedge_transactions_data' # the data set ID

private_key =service_path + service_file

In [ ]:
# pass in our credentials to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [ ]:
# establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

**Build list of owners**

In [ ]:
# query to pull list of owner adn removing card_no 3 since this is for non owners
query = ("""SELECT DISTINCT
(card_no) FROM `wedge2020.wedge_transactions_data.*`
WHERE card_no != 3
GROUP BY card_no
             """)

# execute queries with `client.query`
query_job = client.query(
    query,
    location="US",
)

results = client.query(query)

**Take a sample of the owners**

In [ ]:
sample_of_owners = []

for owner in results:
    sample_of_owners.append(owner[0])
    
    
sample = random.choices(sample_of_owners,k=420) 
# creating sample of 420 owners from 27207 owners from query above
# 420 is an arbritrary number that should get close to the recommended 250 MB

In [ ]:
query = ("""SELECT *
FROM `wedge2020.wedge_transactions_data.*`
WHERE card_no in (
""")

for owner in sample:
    query += str(owner) + ', ' 
query = query[:-2] +')'

In [ ]:
results = client.query(
    query,
    location="US",
)

**Extracts all records associated with those owners**

Adding headers and writing to a new text file named owner_transactions that will contain sample of owners

In [ ]:
headers = ["datetime", "register_no", "emp_no", "trans_no", "upc" , "description", "trans_type", "trans_subtype", "trans_status", "department", "quantity", "Scale", "cost", "unitPrice", "total", "regPrice", "altPrice", "tax", "taxexempt", "foodstamp", "wicable", "discount", "memDiscount", "discountable", "discounttype", "voided", "percentDiscount", "ItemQtty", "volDiscType","volume", "VolSpecial", "mixMatch", "matched", "memType", "staff", "numflag", "itemstatus", "tenderstatus", "charflag", "varflag", "batchHeaderID", "local", "organic", "display", "receipt", "card_no", "store", "branch", "match_id","trans_id"]

In [ ]:
with open("owner_transactions.txt", 'w') as outfile : 
    outfile.write(",".join(headers) + "\n")
    for line in results:
        row = [str(item) for item in line]
        outfile.write(",".join(row) + "\n")